<a href="https://colab.research.google.com/github/Manjiri1101/CMPE255-Manjiri/blob/main/Assignment3/Assignment3_ANN_Manjiri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 3**
## CMPE 255 - Data Mining
## Use state of art libraries for Approximate nearest neighbor search for your favorite data set
## Name:  Manjiri Kadam
### Student ID: 015312076

### **Objective** ✈
Using any data set you want, implement colabs for the following ANN algorithms 
Write proper documentation in read.me file and colabs 

 
1.  LSH

2.  exhaustive search

3. product quantization

4.  trees and graphs

5.  hnsw

In [52]:
!pip install datasketch
!pip install watermark
!pip install mlxtend 
!pip install faiss
# Using rpforest
!pip install rpforest
!pip install faiss-cpu

# 1. LSH

**LSH** stands for Locality-sensitive hashing. This technique hashes similar input items into the same "bucket" with high probability. It is used to find the nearest neighbours

In [53]:

%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
from datasketch import MinHash, MinHashLSHForest

%watermark -a 'Manjiri Kadam' -d -t -v -p numpy,pandas,sklearn,matplotlib

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Author: Manjiri Kadam

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 5.5.0

numpy     : 1.19.5
pandas    : 1.1.5
sklearn   : 0.0
matplotlib: 3.2.2



In [54]:
# Mounting the google drive
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [55]:
#Read the dataset as a dataframe:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/dataset/fox_news.csv')
#Displaying the data:
df.head(5)

,Unnamed: 0,Statement,Link
0,0,Rep. Brady: Inflation as big a concern to Amer...,https://video.foxnews.com/v/6283107807001/
1,1,\nOlympic committee says it spoke with missing...,https://www.foxnews.com/world/peng-shuai-olymp...
2,2,Rep. Brady: Inflation as big a concern to Amer...,https://video.foxnews.com/v/6283107807001/
3,3,"The Blue is back with a brand new, action-pack...",https://nation.foxnews.com/watch/5852616534e7d...
4,4,\nKyle Rittenhouse to appear on 'Tucker Carlso...,https://www.foxnews.com/media/kyle-rittenhouse...


In [56]:
df.sample(2)

,Unnamed: 0,Statement,Link
113,113,Khloe Kardashian slammed for Kyle Rittenhouse ...,https://www.foxnews.com/entertainment/khloe-ka...
92,92,Media malpractice on Kyle Rittenhouse,https://video.foxnews.com/v/6283078170001/


In [57]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [58]:
text = 'The devil went down to Georgia'
print('The shingles (tokens) are:', preprocess(text))

The shingles (tokens) are: ['the', 'devil', 'went', 'down', 'to', 'georgia']


In [59]:
df['shingles'] = df['Statement'].apply(preprocess)

In [60]:
df.sample(2)

,Unnamed: 0,Statement,Link,shingles
19,19,Joy Reid says Kyle Rittenhouse acquittal trace...,https://www.foxnews.com/media/msnbc-joy-reid-k...,"[joy, reid, says, kyle, rittenhouse, acquittal..."
57,57,Political cartoon of the day: Taking notes,https://www.foxnews.com/politics/cartoons-slid...,"[political, cartoon, of, the, day, taking, notes]"


In [61]:
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

In [62]:
def get_forest(data):
    start_time = time.time()
    
    minhash = []
    
    for text in df['Statement']:
        tokens = preprocess(text)
        m = MinHash(num_perm=128)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=128)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [63]:
def predict(text, df, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = df.iloc[idx_array]#['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [64]:
df['forest'] = df['Statement'].apply(get_forest)

It took 0.2246265411376953 seconds to build forest.
It took 0.2023475170135498 seconds to build forest.
It took 0.20916175842285156 seconds to build forest.
It took 0.20604562759399414 seconds to build forest.
It took 0.20813918113708496 seconds to build forest.
It took 0.20472979545593262 seconds to build forest.
It took 0.21049904823303223 seconds to build forest.
It took 0.19683337211608887 seconds to build forest.
It took 0.1992795467376709 seconds to build forest.
It took 0.20030665397644043 seconds to build forest.
It took 0.2006828784942627 seconds to build forest.
It took 0.20059919357299805 seconds to build forest.
It took 0.19954609870910645 seconds to build forest.
It took 0.20637798309326172 seconds to build forest.
It took 0.18982791900634766 seconds to build forest.
It took 0.20895624160766602 seconds to build forest.
It took 0.19698119163513184 seconds to build forest.
It took 0.1928539276123047 seconds to build forest.
It took 0.1908888816833496 seconds to build forest.

In [65]:
df.sample(2)

,Unnamed: 0,Statement,Link,shingles,forest
102,102,Georgia suspect in cop's stabbing is now linke...,https://www.foxnews.com/us/georgia-suspect-in-...,"[georgia, suspect, in, cops, stabbing, is, now...",<datasketch.lshforest.MinHashLSHForest object ...
96,96,Chris Christie on importance of Republicans ge...,https://video.foxnews.com/v/6283074523001/,"[chris, christie, on, importance, of, republic...",<datasketch.lshforest.MinHashLSHForest object ...


In [66]:
forest = get_forest(df)

It took 0.19727063179016113 seconds to build forest.


In [67]:
num_recommendations = 5
title = 'Using a neural net to instantiate a deformable model'
result = predict(title, df, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)
df.head()

It took 0.002725362777709961 seconds to query forest.

 Top Recommendation(s) is(are) 
     Unnamed: 0  ...                                             forest
4            4  ...  <datasketch.lshforest.MinHashLSHForest object ...
72          72  ...  <datasketch.lshforest.MinHashLSHForest object ...
14          14  ...  <datasketch.lshforest.MinHashLSHForest object ...
19          19  ...  <datasketch.lshforest.MinHashLSHForest object ...
62          62  ...  <datasketch.lshforest.MinHashLSHForest object ...

[5 rows x 5 columns]


,Unnamed: 0,Statement,Link,shingles,forest
0,0,Rep. Brady: Inflation as big a concern to Amer...,https://video.foxnews.com/v/6283107807001/,"[rep, brady, inflation, as, big, a, concern, t...",<datasketch.lshforest.MinHashLSHForest object ...
1,1,\nOlympic committee says it spoke with missing...,https://www.foxnews.com/world/peng-shuai-olymp...,"[olympic, committee, says, it, spoke, with, mi...",<datasketch.lshforest.MinHashLSHForest object ...
2,2,Rep. Brady: Inflation as big a concern to Amer...,https://video.foxnews.com/v/6283107807001/,"[rep, brady, inflation, as, big, a, concern, t...",<datasketch.lshforest.MinHashLSHForest object ...
3,3,"The Blue is back with a brand new, action-pack...",https://nation.foxnews.com/watch/5852616534e7d...,"[the, blue, is, back, with, a, brand, new, act...",<datasketch.lshforest.MinHashLSHForest object ...
4,4,\nKyle Rittenhouse to appear on 'Tucker Carlso...,https://www.foxnews.com/media/kyle-rittenhouse...,"[kyle, rittenhouse, to, appear, on, tucker, ca...",<datasketch.lshforest.MinHashLSHForest object ...


#  B) exhaustive search ➿

Exhaustive search is brute force method of finding Aproximate Nearest Neighbour

In [68]:
# Exhaustive search is brute force method.
# Using Housing Boston dataset

column_names = ['Crime', 'ZN', 'INDUS', 'CHAS', 'NOXconc', 'Room', 'AGE', 'DIS', 'RAD', 'Tax', 'PTRatio', 'B', 'LSTAT', 'MedValue']
df2= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/housing.csv', header=None, delimiter=r"\s+", names=column_names)
df2.head(5)

,Crime,ZN,INDUS,CHAS,NOXconc,Room,AGE,DIS,RAD,Tax,PTRatio,B,LSTAT,MedValue
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [69]:
# Including the Sklearn library for exhaustive search
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
# import sklearn.externals as extjoblib
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from sklearn.datasets import load_boston

from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS


In [70]:

boston = load_boston()
X, y = boston.data, boston.target

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [71]:
lr = LinearRegression()

efs = EFS(lr, 
          min_features=10,
          max_features=12,
          scoring='neg_mean_squared_error',
          cv=10)

efs.fit(X, y)

print('Best MSE score: %.2f' % efs.best_score_ * (-1))
print('Best subset:', efs.best_idx_)

Features: 377/377


Best subset: (0, 1, 4, 6, 7, 8, 9, 10, 11, 12)


In [72]:
# http://rasbt.github.io/mlxtend/user_guide/feature_selection/ExhaustiveFeatureSelector/#exhaustive-feature-selector

# c) product quantization ♒
It is a type of ANN . In this method it decompose the space into a Cartesian product of low dimensional subspace and then quantize each subspace separately.

In [73]:
# Implementing Product Quantization using FAISS
# Importing required libraries
import faiss

In [74]:
dimension = 128    # dimensions of each vector                         
n = 256    # number of vectors                   
np.random.seed(1)             
db_vectors = np.random.random((n, dimension)).astype('float32')

In [75]:
m=8
nlist = 5  # number of clusters
quantizer = faiss.IndexFlatL2(dimension)  # coarse quantizer
 
#define the inverted index 
index = faiss.IndexIVFPQ(quantizer, dimension, nlist, m, 8)

In [76]:
# train index on the database vectors:
index.train(db_vectors)
index.add(db_vectors)

In [77]:
nprobe = 2  # find 2 most similar clusters
n_query = 10
k = 3  # return 3 nearest neighbours,
np.random.seed(0)   
query_vectors = np.random.random((n_query, dimension)).astype('float32')
distances, indices = index.search(query_vectors, k)
print(distances)
print(indices)

[[15.95562  15.998074 16.059832]
 [16.276657 18.102848 18.522284]
 [15.520997 16.500257 16.77318 ]
 [18.64824  18.720179 18.868248]
 [17.3755   18.019579 18.28939 ]
 [16.375307 17.643938 17.803608]
 [14.23982  15.031904 15.587464]
 [16.380919 16.686506 17.270594]
 [16.17979  18.766174 18.855637]
 [17.293684 17.385288 17.451862]]
[[ 19   4 245]
 [  6 110  51]
 [148 149 216]
 [117  29  27]
 [ 82 196  16]
 [ 51 141 127]
 [ 16 196  60]
 [ 12 216  46]
 [142  59 248]
 [245  46  19]]


# d) trees and graphs#
For the assignment, used annoy library by Spotify. It is the fastest library for nearest neighbour search. 

In [78]:
!pip install annoy

In [79]:
X = df['Statement']

In [80]:
# from rpforest import RPForest

# model = RPForest(leaf_size=50, no_trees=10)
# model.fit(X)

In [81]:
import pickle
import annoy

In [82]:
def load_data():
    with open('/content/gdrive/MyDrive/Data_Mining/movies.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data


{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

In [83]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [84]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [85]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Aladdin (1992)
* Blade Runner (1982)
* Aliens (1986)
* Pink Floyd - The Wall (1982)
* Brazil (1985)
* Wrong Trousers, The (1993)
* Alien (1979)
* Return of the Jedi (1983)
* E.T. the Extra-Terrestrial (1982)


# e) hnsw

In [86]:
!pip install nmslib

In [87]:
import nmslib

In [88]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [89]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

In [90]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Beauty and the Beast (1991)
* Fantasia (1940)
* Heavy Metal (1981)
* Aladdin (1992)
* Snow White and the Seven Dwarfs (1937)
* Batman (1989)
* James and the Giant Peach (1996)
* Blade Runner (1982)
* Aliens (1986)


### References▶
1.  https://towardsdatascience.com/locality-sensitive-hashing-in-nlp-1fb3d4a7ba9f
2.  https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/
3. http://ethen8181.github.io/machine-learning/recsys/content_based/lsh_text.html
4. https://github.com/eyaltrabelsi/my-notebooks/tree/master/Lectures/search_in_practice-approximate_nearest_neighbors